# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of each sentence from `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = np.array([[source_vocab_to_int[word] for word in row.split()] for row in source_text.split('\n')])
    target_id_text = np.array([[target_vocab_to_int[word] for word in row.split()] + [target_vocab_to_int['<EOS>']] for row in target_text.split('\n')])
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.

Return the placeholders in the following the tuple (Input, Targets, Learing Rate, Keep Probability)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate, keep probability)
    """
    inputs = tf.placeholder(tf.int32, shape=[None, None], name='input')
    targets = tf.placeholder(tf.int32, shape=[None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, learning_rate, keep_prob

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoding Input
Implement `process_decoding_input` using TensorFlow to remove the last word id from each batch in `target_data` and concat the GO ID to the beginning of each batch.

In [8]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for dencoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    ending = tf.strided_slice(target_data, begin=[0, 0], end=[batch_size, -1], strides=[1, 1])
    return tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_decoding_input(process_decoding_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer using [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn).

In [9]:
import math

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :return: RNN state
    """
    with tf.name_scope('Encoder'):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.truncated_normal_initializer(stddev=(1/math.sqrt(rnn_size))))
        cell = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        rnn_cells = tf.contrib.rnn.MultiRNNCell([lstm] * num_layers)

        rnn_outputs, rnn_final_state = tf.nn.dynamic_rnn(rnn_cells, rnn_inputs, dtype=tf.float32)
    
    return rnn_final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder).  Apply the `output_fn` to the [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) outputs.

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """
    dynamic_fn_train = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    outputs, _, _  = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell,
                                                            decoder_fn=dynamic_fn_train,
                                                            inputs=dec_embed_input,
                                                            sequence_length=sequence_length,
                                                            scope=decoding_scope)
    logits = output_fn(outputs)
    
    return logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    # TODO: Implement Function
    dynamic_fn_inference = tf.contrib.seq2seq.simple_decoder_fn_inference(output_fn, 
                                                                          encoder_state,
                                                                          dec_embeddings,
                                                                          start_of_sequence_id,
                                                                          end_of_sequence_id,
                                                                          maximum_length,
                                                                          num_decoder_symbols=vocab_size)
    outputs, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell,
                                                           decoder_fn=dynamic_fn_inference,
                                                           scope=decoding_scope)
    
    return outputs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
import math

def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    """
    Create decoding layer
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    initializer = lambda size: tf.truncated_normal_initializer(mean=0.0, stddev=(1/math.sqrt(size)))
    
    with tf.variable_scope('Decoder') as decoding_scope:
        # Construct the decoder cell
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=initializer(rnn_size))
        dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([dec_cell] * num_layers)

        # Lambda function to generate the output
        output_fn = lambda x: tf.contrib.layers.fully_connected(
                                                    x,
                                                    num_outputs=vocab_size,
                                                    weights_initializer=initializer(vocab_size),
                                                    scope=decoding_scope)
        # Set up the training decoder
        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob)
        
    with tf.variable_scope('Decoder', reuse=True) as decoding_scope:        
        # Set up the inference decoder
        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']
        
        inference_logits = decoding_layer_infer(encoder_state,
                                                dec_cell,
                                                dec_embeddings,
                                                start_of_sequence_id,
                                                end_of_sequence_id,
                                                sequence_length,
                                                vocab_size,
                                                decoding_scope,
                                                output_fn,
                                                keep_prob)
    
    return train_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)`.
- Process target data using your `process_decoding_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)`.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param sequence_length: Sequence Length
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training Logits, Inference Logits)
    """
    
    with tf.name_scope('Embedding'):
        rnn_inputs = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size, 
                                                       initializer=tf.truncated_normal_initializer(stddev=(1/math.sqrt(source_vocab_size))))
    
    encod_state = encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)
    
    decod_input = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    decod_embedding = tf.Variable(tf.truncated_normal(shape=[target_vocab_size, dec_embedding_size], stddev=(1/math.sqrt(target_vocab_size))))
    decod_embed = tf.nn.embedding_lookup(decod_embedding, decod_input)
    
    train_logits, infer_logits = decoding_layer(decod_embed, decod_embedding, encod_state, target_vocab_size,
                                                sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)
    
    return train_logits, infer_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability

In [14]:
# Number of Epochs
epochs = 300
# Batch Size
batch_size = 4098
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 64
decoding_embedding_size = 64
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    0/33 - Train Accuracy:  0.318, Validation Accuracy:  0.317, Loss:  5.881
Epoch   0 Batch    1/33 - Train Accuracy:  0.335, Validation Accuracy:  0.336, Loss:  5.849
Epoch   0 Batch    2/33 - Train Accuracy:  0.324, Validation Accuracy:  0.324, Loss:  5.757
Epoch   0 Batch    3/33 - Train Accuracy:  0.276, Validation Accuracy:  0.306, Loss:  5.529
Epoch   0 Batch    4/33 - Train Accuracy:  0.308, Validation Accuracy:  0.306, Loss:  5.018
Epoch   0 Batch    5/33 - Train Accuracy:  0.310, Validation Accuracy:  0.306, Loss:  4.562
Epoch   0 Batch    6/33 - Train Accuracy:  0.307, Validation Accuracy:  0.306, Loss:  4.278
Epoch   0 Batch    7/33 - Train Accuracy:  0.318, Validation Accuracy:  0.317, Loss:  4.104
Epoch   0 Batch    8/33 - Train Accuracy:  0.339, Validation Accuracy:  0.338, Loss:  3.977
Epoch   0 Batch    9/33 - Train Accuracy:  0.311, Validation Accuracy:  0.340, Loss:  4.001
Epoch   0 Batch   10/33 - Train Accuracy:  0.342, Validation Accuracy:  0.340, L

Epoch   2 Batch   26/33 - Train Accuracy:  0.475, Validation Accuracy:  0.471, Loss:  2.506
Epoch   2 Batch   27/33 - Train Accuracy:  0.517, Validation Accuracy:  0.471, Loss:  2.273
Epoch   2 Batch   28/33 - Train Accuracy:  0.454, Validation Accuracy:  0.479, Loss:  2.580
Epoch   2 Batch   29/33 - Train Accuracy:  0.495, Validation Accuracy:  0.468, Loss:  2.340
Epoch   2 Batch   30/33 - Train Accuracy:  0.470, Validation Accuracy:  0.467, Loss:  2.409
Epoch   2 Batch   31/33 - Train Accuracy:  0.477, Validation Accuracy:  0.479, Loss:  2.403
Epoch   3 Batch    0/33 - Train Accuracy:  0.480, Validation Accuracy:  0.479, Loss:  2.367
Epoch   3 Batch    1/33 - Train Accuracy:  0.471, Validation Accuracy:  0.470, Loss:  2.337
Epoch   3 Batch    2/33 - Train Accuracy:  0.478, Validation Accuracy:  0.479, Loss:  2.325
Epoch   3 Batch    3/33 - Train Accuracy:  0.455, Validation Accuracy:  0.479, Loss:  2.411
Epoch   3 Batch    4/33 - Train Accuracy:  0.478, Validation Accuracy:  0.473, L

Epoch   5 Batch   20/33 - Train Accuracy:  0.463, Validation Accuracy:  0.489, Loss:  1.878
Epoch   5 Batch   21/33 - Train Accuracy:  0.496, Validation Accuracy:  0.494, Loss:  1.787
Epoch   5 Batch   22/33 - Train Accuracy:  0.546, Validation Accuracy:  0.499, Loss:  1.616
Epoch   5 Batch   23/33 - Train Accuracy:  0.471, Validation Accuracy:  0.496, Loss:  1.861
Epoch   5 Batch   24/33 - Train Accuracy:  0.466, Validation Accuracy:  0.491, Loss:  1.853
Epoch   5 Batch   25/33 - Train Accuracy:  0.497, Validation Accuracy:  0.494, Loss:  1.745
Epoch   5 Batch   26/33 - Train Accuracy:  0.500, Validation Accuracy:  0.495, Loss:  1.733
Epoch   5 Batch   27/33 - Train Accuracy:  0.538, Validation Accuracy:  0.492, Loss:  1.600
Epoch   5 Batch   28/33 - Train Accuracy:  0.471, Validation Accuracy:  0.489, Loss:  1.821
Epoch   5 Batch   29/33 - Train Accuracy:  0.515, Validation Accuracy:  0.488, Loss:  1.651
Epoch   5 Batch   30/33 - Train Accuracy:  0.490, Validation Accuracy:  0.487, L

Epoch   8 Batch   14/33 - Train Accuracy:  0.479, Validation Accuracy:  0.503, Loss:  1.440
Epoch   8 Batch   15/33 - Train Accuracy:  0.495, Validation Accuracy:  0.515, Loss:  1.442
Epoch   8 Batch   16/33 - Train Accuracy:  0.486, Validation Accuracy:  0.505, Loss:  1.437
Epoch   8 Batch   17/33 - Train Accuracy:  0.506, Validation Accuracy:  0.506, Loss:  1.375
Epoch   8 Batch   18/33 - Train Accuracy:  0.511, Validation Accuracy:  0.508, Loss:  1.375
Epoch   8 Batch   19/33 - Train Accuracy:  0.506, Validation Accuracy:  0.505, Loss:  1.371
Epoch   8 Batch   20/33 - Train Accuracy:  0.473, Validation Accuracy:  0.498, Loss:  1.439
Epoch   8 Batch   21/33 - Train Accuracy:  0.506, Validation Accuracy:  0.502, Loss:  1.372
Epoch   8 Batch   22/33 - Train Accuracy:  0.546, Validation Accuracy:  0.501, Loss:  1.246
Epoch   8 Batch   23/33 - Train Accuracy:  0.474, Validation Accuracy:  0.495, Loss:  1.433
Epoch   8 Batch   24/33 - Train Accuracy:  0.476, Validation Accuracy:  0.498, L

Epoch  11 Batch    8/33 - Train Accuracy:  0.516, Validation Accuracy:  0.510, Loss:  1.200
Epoch  11 Batch    9/33 - Train Accuracy:  0.506, Validation Accuracy:  0.526, Loss:  1.253
Epoch  11 Batch   10/33 - Train Accuracy:  0.531, Validation Accuracy:  0.536, Loss:  1.200
Epoch  11 Batch   11/33 - Train Accuracy:  0.492, Validation Accuracy:  0.511, Loss:  1.251
Epoch  11 Batch   12/33 - Train Accuracy:  0.486, Validation Accuracy:  0.511, Loss:  1.250
Epoch  11 Batch   13/33 - Train Accuracy:  0.537, Validation Accuracy:  0.536, Loss:  1.191
Epoch  11 Batch   14/33 - Train Accuracy:  0.493, Validation Accuracy:  0.516, Loss:  1.232
Epoch  11 Batch   15/33 - Train Accuracy:  0.504, Validation Accuracy:  0.523, Loss:  1.235
Epoch  11 Batch   16/33 - Train Accuracy:  0.480, Validation Accuracy:  0.498, Loss:  1.254
Epoch  11 Batch   17/33 - Train Accuracy:  0.499, Validation Accuracy:  0.498, Loss:  1.260
Epoch  11 Batch   18/33 - Train Accuracy:  0.521, Validation Accuracy:  0.515, L

...

Epoch 286 Batch   28/33 - Train Accuracy:  0.871, Validation Accuracy:  0.875, Loss:  0.227
Epoch 286 Batch   29/33 - Train Accuracy:  0.880, Validation Accuracy:  0.873, Loss:  0.208
Epoch 286 Batch   30/33 - Train Accuracy:  0.874, Validation Accuracy:  0.873, Loss:  0.217
Epoch 286 Batch   31/33 - Train Accuracy:  0.879, Validation Accuracy:  0.874, Loss:  0.214
Epoch 287 Batch    0/33 - Train Accuracy:  0.873, Validation Accuracy:  0.876, Loss:  0.217
Epoch 287 Batch    1/33 - Train Accuracy:  0.885, Validation Accuracy:  0.878, Loss:  0.207
Epoch 287 Batch    2/33 - Train Accuracy:  0.878, Validation Accuracy:  0.879, Loss:  0.212
Epoch 287 Batch    3/33 - Train Accuracy:  0.881, Validation Accuracy:  0.879, Loss:  0.218
Epoch 287 Batch    4/33 - Train Accuracy:  0.883, Validation Accuracy:  0.878, Loss:  0.210
Epoch 287 Batch    5/33 - Train Accuracy:  0.881, Validation Accuracy:  0.877, Loss:  0.211
Epoch 287 Batch    6/33 - Train Accuracy:  0.878, Validation Accuracy:  0.876, L

Epoch 289 Batch   22/33 - Train Accuracy:  0.885, Validation Accuracy:  0.875, Loss:  0.200
Epoch 289 Batch   23/33 - Train Accuracy:  0.868, Validation Accuracy:  0.876, Loss:  0.228
Epoch 289 Batch   24/33 - Train Accuracy:  0.869, Validation Accuracy:  0.875, Loss:  0.229
Epoch 289 Batch   25/33 - Train Accuracy:  0.886, Validation Accuracy:  0.877, Loss:  0.211
Epoch 289 Batch   26/33 - Train Accuracy:  0.880, Validation Accuracy:  0.877, Loss:  0.208
Epoch 289 Batch   27/33 - Train Accuracy:  0.886, Validation Accuracy:  0.876, Loss:  0.202
Epoch 289 Batch   28/33 - Train Accuracy:  0.872, Validation Accuracy:  0.875, Loss:  0.227
Epoch 289 Batch   29/33 - Train Accuracy:  0.881, Validation Accuracy:  0.875, Loss:  0.208
Epoch 289 Batch   30/33 - Train Accuracy:  0.874, Validation Accuracy:  0.874, Loss:  0.217
Epoch 289 Batch   31/33 - Train Accuracy:  0.878, Validation Accuracy:  0.873, Loss:  0.214
Epoch 290 Batch    0/33 - Train Accuracy:  0.876, Validation Accuracy:  0.874, L

Epoch 292 Batch   16/33 - Train Accuracy:  0.870, Validation Accuracy:  0.876, Loss:  0.230
Epoch 292 Batch   17/33 - Train Accuracy:  0.883, Validation Accuracy:  0.875, Loss:  0.208
Epoch 292 Batch   18/33 - Train Accuracy:  0.880, Validation Accuracy:  0.877, Loss:  0.215
Epoch 292 Batch   19/33 - Train Accuracy:  0.877, Validation Accuracy:  0.876, Loss:  0.214
Epoch 292 Batch   20/33 - Train Accuracy:  0.870, Validation Accuracy:  0.873, Loss:  0.223
Epoch 292 Batch   21/33 - Train Accuracy:  0.880, Validation Accuracy:  0.874, Loss:  0.219
Epoch 292 Batch   22/33 - Train Accuracy:  0.884, Validation Accuracy:  0.875, Loss:  0.201
Epoch 292 Batch   23/33 - Train Accuracy:  0.871, Validation Accuracy:  0.877, Loss:  0.229
Epoch 292 Batch   24/33 - Train Accuracy:  0.869, Validation Accuracy:  0.876, Loss:  0.229
Epoch 292 Batch   25/33 - Train Accuracy:  0.884, Validation Accuracy:  0.874, Loss:  0.211
Epoch 292 Batch   26/33 - Train Accuracy:  0.879, Validation Accuracy:  0.873, L

Epoch 295 Batch   10/33 - Train Accuracy:  0.874, Validation Accuracy:  0.877, Loss:  0.216
Epoch 295 Batch   11/33 - Train Accuracy:  0.871, Validation Accuracy:  0.876, Loss:  0.232
Epoch 295 Batch   12/33 - Train Accuracy:  0.874, Validation Accuracy:  0.876, Loss:  0.221
Epoch 295 Batch   13/33 - Train Accuracy:  0.876, Validation Accuracy:  0.876, Loss:  0.217
Epoch 295 Batch   14/33 - Train Accuracy:  0.874, Validation Accuracy:  0.874, Loss:  0.222
Epoch 295 Batch   15/33 - Train Accuracy:  0.875, Validation Accuracy:  0.876, Loss:  0.218
Epoch 295 Batch   16/33 - Train Accuracy:  0.873, Validation Accuracy:  0.878, Loss:  0.230
Epoch 295 Batch   17/33 - Train Accuracy:  0.882, Validation Accuracy:  0.876, Loss:  0.208
Epoch 295 Batch   18/33 - Train Accuracy:  0.880, Validation Accuracy:  0.877, Loss:  0.214
Epoch 295 Batch   19/33 - Train Accuracy:  0.877, Validation Accuracy:  0.877, Loss:  0.215
Epoch 295 Batch   20/33 - Train Accuracy:  0.870, Validation Accuracy:  0.874, L

Epoch 298 Batch    4/33 - Train Accuracy:  0.880, Validation Accuracy:  0.874, Loss:  0.210
Epoch 298 Batch    5/33 - Train Accuracy:  0.877, Validation Accuracy:  0.874, Loss:  0.211
Epoch 298 Batch    6/33 - Train Accuracy:  0.878, Validation Accuracy:  0.875, Loss:  0.217
Epoch 298 Batch    7/33 - Train Accuracy:  0.879, Validation Accuracy:  0.876, Loss:  0.214
Epoch 298 Batch    8/33 - Train Accuracy:  0.876, Validation Accuracy:  0.875, Loss:  0.216
Epoch 298 Batch    9/33 - Train Accuracy:  0.870, Validation Accuracy:  0.876, Loss:  0.226
Epoch 298 Batch   10/33 - Train Accuracy:  0.875, Validation Accuracy:  0.876, Loss:  0.216
Epoch 298 Batch   11/33 - Train Accuracy:  0.870, Validation Accuracy:  0.875, Loss:  0.231
Epoch 298 Batch   12/33 - Train Accuracy:  0.874, Validation Accuracy:  0.876, Loss:  0.220
Epoch 298 Batch   13/33 - Train Accuracy:  0.877, Validation Accuracy:  0.876, Loss:  0.217
Epoch 298 Batch   14/33 - Train Accuracy:  0.875, Validation Accuracy:  0.876, L

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [21]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return [vocab_to_int[word] if word in vocab_to_int.keys() else vocab_to_int['<UNK>'] for word in sentence.lower().split()]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [22]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

Input
  Word Ids:      [180, 132, 200, 157, 214, 23, 71]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [298, 184, 246, 334, 143, 197, 92, 304, 1]
  French Words: ['il', 'a', 'conduit', 'un', 'vieux', 'camion', 'jaune', '.', '<EOS>']


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.